In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import math

# Inicia a contagem de tempo
start_time = time.time()

# Valores iniciais físicos
length = 1
length_x = 0
length_x_delta = 0.1
time_step = 0

time_step_delta = 0.025

delta_t_presented = 10000

time_max = 12500

steps_length = int((length / length_x_delta) + 1)
steps_time = int((time_max / time_step_delta) + 1)

index_x = 0
index_t = 0
m = 0
m_max = 100
thermal_conductivity_t = 1
mass_density = 1
thermal_capacity_t = 1
thermal_diffusivity_t = thermal_conductivity_t / (mass_density * thermal_capacity_t)
matrix_temperature = np.zeros((steps_length, steps_time))
initial_temperature = 0

boundary_temperature_1 = 200
boundary_temperature_2 = 100

beta_1 = 1

beta_2 = 0
s = 1
# s = 1/6

# s = (thermal_diffusivity_t * time_step_delta) / (length_x_delta * length_x_delta)

# Equação de T com o somatório (Solução Analítica)
i1 = 0
sum_temperature = 0

while m < steps_time:
    matrix_temperature[0][m] = boundary_temperature_1
    matrix_temperature[steps_length - 1][m] = boundary_temperature_2
    m = m + 1

m = 0

while index_t <= steps_time:
    while index_x < steps_length:
        while m <= m_max:
            beta_2 = ((m + 1) * math.pi) / length
            sum_temperature = sum_temperature + (math.pow((math.e), (-thermal_diffusivity_t * beta_2 * beta_2 * time_step))) * ((math.sin(beta_2 * length_x)) / (beta_2)) * ((boundary_temperature_2 - initial_temperature) * (math.cos(beta_2 * length)) - (boundary_temperature_1 - initial_temperature))
            m = m + 1

        matrix_temperature[index_x][index_t] = boundary_temperature_1 + ((boundary_temperature_2 - boundary_temperature_1) * (length_x / length)) + (2 / length) * sum_temperature
        length_x = length_x + length_x_delta
        index_x = index_x + 1
        m = 0

    time_step = time_step + time_step_delta
    index_t = index_t + 1
    sum_temperature = 0

m = 1
time_step = 0
length_x = 0
index_t = 0
index_x = 0

# Solução Numérica
numeric_matrix = np.zeros((steps_time, steps_length))
big_j = steps_length - 1
p1 = np.zeros(big_j + 1)
q1 = np.zeros(big_j + 1)
p1[0] = 0
q1[0] = boundary_temperature_1
aj = 0
bj = 0
cj = 0
dj = 0
n = 0
j = 1

while n < steps_time:
    numeric_matrix[n][0] = boundary_temperature_1
    numeric_matrix[n][big_j] = boundary_temperature_2
    n = n + 1

n = 0

# Algoritmo de Thomas (TDMA)
while n < steps_time - 1:
    j = 1
    p1[0] = 0
    q1[0] = boundary_temperature_1
    p1[big_j] = 0
    q1[big_j] = boundary_temperature_2
    while j <= big_j - 1:
        aj = 1 + (2 * s) * (1 - beta_1)
        bj = -(s * (beta_1 - 1))
        cj = -(s * (beta_1 - 1))
        dj = s * (numeric_matrix[n][j - 1]) + (1 - 2 * s * beta_1) * (numeric_matrix[n][j]) + (s * beta_1 * numeric_matrix[n][j + 1])
        p1[j] = bj / (aj - cj * p1[j - 1])
        q1[j] = (dj + cj * q1[j - 1]) / (aj - cj * p1[j - 1])
        j = j + 1
    j = big_j - 1
    while j >= 0:
        numeric_matrix[n + 1][j] = p1[j] * numeric_matrix[n + 1][j + 1] + q1[j]
        j = j - 1
    n = n + 1

sum_rms = 0
m = 1

while m < big_j + 1:
    sum_rms = sum_rms + math.pow((numeric_matrix[steps_time - 1][m] - matrix_temperature[steps_length - 1][steps_time - 1]), 2)
    m = m + 1

rms = math.sqrt(sum_rms / (big_j + 1))
t_presented = delta_t_presented
steps_x = (steps_length - 1) / 10

if beta_1 == 1:
    print('Equação de Difusão: Formulação Explícita')
if beta_1 == 1 / 2:
    print('Equação de Difusão: Esquema de Crank-Nicolson')
if beta_1 == 0:
    print('Equação de Difusão: Formulação Totalmente Implícita')

print('\n')
print('JMAX = ', big_j, '  TMAX = ', time_max, '  T1 = ', boundary_temperature_1, '  T2 = ', boundary_temperature_2,
      '  s = ', s)
print('\n')
print('ALPH = ', thermal_diffusivity_t, '  BETA = ', beta_1, '  DELT_T = ', time_step_delta, '  DELT_X = ', length_x_delta)
#print('\n')

while t_presented < steps_time:
    print('\n')
    print('t = ', (time_step_delta * t_presented), '  TN = ', numeric_matrix[t_presented][0], '  ',
          numeric_matrix[t_presented][int(steps_x * 1)], '  ', numeric_matrix[t_presented][int(steps_x * 2)], '  ',
          numeric_matrix[t_presented][int(steps_x * 3)], '  ', numeric_matrix[t_presented][int(steps_x * 4)], '  ',
          numeric_matrix[t_presented][int(steps_x * 5)], '  ', numeric_matrix[t_presented][int(steps_x * 6)], '  ',
          numeric_matrix[t_presented][int(steps_x * 7)], '  ', numeric_matrix[t_presented][int(steps_x * 8)], '  ',
          numeric_matrix[t_presented][int(steps_x * 9)], '  ', numeric_matrix[t_presented][int(steps_x * 10)])
    t_presented = t_presented + delta_t_presented

t_presented = t_presented - delta_t_presented
print('\n\n')
print('t = ', (time_step_delta * t_presented), '  TE = ', matrix_temperature[0][t_presented], '  ',
      matrix_temperature[int(steps_x * 1)][t_presented], '  ', matrix_temperature[int(steps_x * 2)][t_presented], '  ',
      matrix_temperature[int(steps_x * 3)][t_presented], '  ', matrix_temperature[int(steps_x * 4)][t_presented], '  ',
      matrix_temperature[int(steps_x * 5)][t_presented], '  ', matrix_temperature[int(steps_x * 6)][t_presented], '  ',
      matrix_temperature[int(steps_x * 7)][t_presented], '  ', matrix_temperature[int(steps_x * 8)][t_presented], '  ',
      matrix_temperature[int(steps_x * 9)][t_presented], '  ', matrix_temperature[int(steps_x * 10)][t_presented])

print('\n')
print('RMS = ', rms)


<ipython-input-2-1a4a6fb8a4ed>:111: RuntimeWarning: overflow encountered in double_scalars
  dj = s * (numeric_matrix[n][j - 1]) + (1 - 2 * s * beta_1) * (numeric_matrix[n][j]) + (s * beta_1 * numeric_matrix[n][j + 1])
<ipython-input-2-1a4a6fb8a4ed>:113: RuntimeWarning: invalid value encountered in double_scalars
  q1[j] = (dj + cj * q1[j - 1]) / (aj - cj * p1[j - 1])


Equação de Difusão: Formulação Explícita


JMAX =  10   TMAX =  12500   T1 =  200   T2 =  100   s =  1


ALPH =  1.0   BETA =  1   DELT_T =  0.025   DELT_X =  0.1


t =  250.0   TN =  nan    nan    nan    nan    nan    nan    nan    nan    nan    nan    100.0


t =  500.0   TN =  nan    nan    nan    nan    nan    nan    nan    nan    nan    nan    100.0


t =  750.0   TN =  nan    nan    nan    nan    nan    nan    nan    nan    nan    nan    100.0


t =  1000.0   TN =  nan    nan    nan    nan    nan    nan    nan    nan    nan    nan    100.0


t =  1250.0   TN =  nan    nan    nan    nan    nan    nan    nan    nan    nan    nan    100.0


t =  1500.0   TN =  nan    nan    nan    nan    nan    nan    nan    nan    nan    nan    100.0


t =  1750.0   TN =  nan    nan    nan    nan    nan    nan    nan    nan    nan    nan    100.0


t =  2000.0   TN =  nan    nan    nan    nan    nan    nan    nan    nan    nan    nan    100.0


t =  2250.0   TN =  nan    nan    nan    nan    nan   